<div align="center">
<a href="https://aiblog.nl"><img src=".\images\aiblog_logo.png" width="400"></a>
</div>

# ELIZA

ELIZA is een bekend AI programma dat in 1964-1966 ontwikkeld werd door MIT professor Joseph Weizenbaum. ELIZA gedraagt zich als een rogeriaanse psycholoog. Ze luistert aktief naar wat een gebruiker zegt en reageert hierop in het Engels door vragen stellen.

Tegenwoordig noemen we ELIZA een 'chatbot'. Chatbots krijgen weer volop aandacht omdat met deep learning AI technieken nieuwe resultaten worden behaald. Chatbots worden ook wel de 'nieuwe apps' genoemd. In combinatie met spraakgestuurde apparaten zoals Google Home, iPhone/HomePod (Siri) en Amazon Echo (Alexa) kan deze technologie erg bruikbaar worden.

De naam ELIZA komt uit het toneelstuk Pygmalion (1913). In het toneelstuk gaat professor Higgings de weddenschap aan dat hij de eenvoudige bloemenverkoopstertje Eliza Doolittle in korte tijd Engels kan leren.

Het onderzoek naar ELIZA maakt duidelijk dat een programma dat intelligent gedrag nabootst, zelf niet intelligent hoeft te zijn. Vergelijkbaar met het onderzoek naar de General Problem Solver heeft ELIZA de ontwikkeling van technologie gestimuleerd. ELIZA maakt gebruik van patroonvergelijking (engels: 'pattern matching'), een onmisbare component in veel computertalen en programma's.

Hieronder wordt een ELIZA programma in Python beschreven. De beschrijving volgt in grote lijnen het boek <a href="https://github.com/norvig/paip-lisp">'Paradigms of Artificial Intelligence Programming' van Peter Norvig (1991)</a> maar gebruikt Python i.p.v. Lisp. 


# Principe

ELIZA werkt volgens een eenvoudig principe. Een gebruiker voert een zin in en ELIZA reageert hierop. De reactie wordt bepaald door de invoer van de gebruiker te vergelijken met een grote lijst van invoer-reactie mogelijkheden. Een invoer-reactie combinatie wordt een regel (Engels: 'rule') genoemd. Als de invoer van de gebruiker in de lijst van regels voorkomt, dan reageert ELIZA met de bijbehorende reactie. Als de invoer niet voorkomt, dan zal ELIZA met een standaard reactie reageren.

In [388]:
# Een dictionary is een handige data structuur in Python om 
# rules in op te slaan.
rules = {
    'hi': 'hi',
    'my name is albert': 'nice to meet you albert',
    'what is your name': 'my name is ELIZA',
    '[exit]': 'bye bye',
}

def eliza(rules):
    input_ = ''
    while input_ != '[exit]':
        input_ = input('you> ')
        if input_ in rules.keys():
            response = rules[input_]
        else:
            response = 'I do not understand'
        print('eliza> {}'.format(response))

eliza(rules)
# type '[exit]' om door te gaan met de executie van deze notebook!

you> [exit]
eliza> bye bye


<div align="center">
<img src=".\images\eliza01.png" width="100%">
</div>

In een paar regels code kan je de essentie van ELIZA programmeren. Als je het aantal rules verhoogd dan lijkt ELIZA steeds 'slimmer' te worden. Een geloofwaardige chatbot heeft dan ook een grote set van regels, zodat het kan reageren op veel verschillende situaties. 

Het voorbeeld laat ook een tekortkoming zien van deze aanpak. Het is namelijk niet efficient om voor elke naam de regel `'my name is ...' -> 'nice to meet you ...'` in te voeren. Het is beter om hiervoor gebruik te maken van variabelen.

Een variable in een regel wordt aangeduid met het symbool '?'. De regel

`'my name is ?x' -> 'nice to meet you ?x'`

kan omgaan met alle namen. Het eerste gedeelte van deze regel noemen we `pattern` en het tweede gedeelte `response`:

`pattern -> response`.

Om gebruik te kunnen maken van variabelen is een patroonherkenningstechniek nodig. ELIZA wordt hierdoor iets complexer, maar daardoor wel krachtiger. De basis van deze techniek is het ontleden van de invoer van een gebruiker en de pattern van regels in atomaire eenheden (ook wel 'tokens' genoemd). Dit kunnen woorden of variablen zijn. Vervolgens wordt elke eenheid uit de invoer en het pattern met elkaar vergeleken. Woorden moeten altijd aan elkaar gelijk zijn. Een variable in een pattern kan gelijk zijn aan elk woord.

In [390]:
# Invoer en patronen worden opgeslagen in lijsten van basis
# eenheden (woorden, variabelen). In Python kan je efficient
# met lijsten omgaan. Om de code duidelijk te houden definieren
# we enkele functies.

def atom(x):
    """Test of x een enkel item is."""
    return len(x)==1

def first(x):
    """Eerste element van een lijst."""
    return x[0:1]

def rest(x):
    """Alle elementen behalve de eerste van een lijst."""
    return x[1:]

def simple_equal(pattern, input_):
    """Test of x en y gelijk zijn. Functie recursie worden gebruikt."""
    if atom(pattern) or atom(input_):
        return pattern == input_
    return simple_equal(first(pattern), first(input_)) and \
           simple_equal(rest(pattern), rest(input_))    
    
x = "hello my name is albert".split() # gebruik 'split' om van een string een lijst te maken
y = "hello my name is albert".split()
print("{} == {} is {}".format(x, y, simple_equal(x, y)))

x = "hello my name is albert".split()
y = "hello my name is john".split()
print("{} == {} is {}".format(x, y, simple_equal(x, y)))

['hello', 'my', 'name', 'is', 'albert'] == ['hello', 'my', 'name', 'is', 'albert'] is True
['hello', 'my', 'name', 'is', 'albert'] == ['hello', 'my', 'name', 'is', 'john'] is False


De volgende stap is het herkennen van variabelen in een pattern. Een variabele is een atomaire eenheid dat begint met het symbool '?'. Zodra een variabele wordt herkent in een pattern dan is de match met de eenheid uit de invoer altijd waar.

In [391]:
def variable_p(x):
    """Test of x een variabele is (een symbool dat begint met '?')?"""
    return x[0][0] == '?'
    
def pat_match (pattern, input_):
    """Test of x en y gelijk zijn. Een variabele is gelijk aan alles."""
    # Opmerking: omdat 'input' een functie is in Python voegen we '_' toe
    # aan 'input' om deze naam als variabele naam te gebruiken
    if variable_p(pattern):
        return True
    if atom(pattern) or atom(input_):
        return pattern == input_
    return pat_match(first(pattern), first(input_)) and \
           pat_match(rest(pattern), rest(input_))

pattern = "hello my name is ?x".split()
input_  = "hello my name is albert".split()
print("{} == {} is {}".format(pattern, input_, pat_match(pattern, input_)))

['hello', 'my', 'name', 'is', '?x'] == ['hello', 'my', 'name', 'is', 'albert'] is True


# Complexe patroonherkenning

In het zojuist beschreven voorbeeld wordt de waarde die aan een variabele wordt toegekend niet gebruik. Hierdoor zal het systeem verkeerd reageren op de volgende situatie:

In [392]:
pattern = "?x is ?x".split()
input_  = "albert is john".split()
print("{} == {} is {}".format(pattern, input_, pat_match(pattern, input_)))

['?x', 'is', '?x'] == ['albert', 'is', 'john'] is True


Om dit goed te doen met de toekenning van een waarde aan een variabele opgeslagen worden. Dit wordt 'binding' genoemd. Een bindings-lijst heeft twee functies. Teneerste kan je binnen een pattern controleren of de variabele geen dubbele waarden krijgt. Ten tweede kan je de waarden van de variabelen gebruiken in de reactie die ELIZA gaat geven.

In [393]:
def consp(x):
    """Test of list tenminste 1 element heeft."""
    return len(x)>0

def pat_match(pattern, input_, bindings={}):
    if bindings==None:
        return None
    elif variable_p(pattern):
        return match_variable(pattern[0], input_[0], bindings)
    elif pattern == input_:
        return bindings
    elif atom(pattern) or atom(input_):
        return bindings
    elif consp(pattern) and consp(input_):
        return pat_match(rest(pattern), rest(input_), 
                         pat_match(first(pattern), first(input_), bindings))
    else:
        return None
    
def match_variable(var, input_, bindings):
    """Bind een variable of test of de waarde gelijk is aan de input."""
    if var not in bindings.keys():
        bindings[var] = input_
        return bindings
    elif input_ == bindings[var]:
        return bindings
    return None

pattern = '?x is ?x'.split()
input_  = 'albert is albert'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

pattern = '?x is ?x'.split()
input_  = 'albert is john'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

['?x', 'is', '?x'] == ['albert', 'is', 'albert'] is True
Bindings = {'?x': 'albert'}
['?x', 'is', '?x'] == ['albert', 'is', 'john'] is True
Bindings = {'?x': 'albert'}


In [394]:
def sublis(bindings, response):
    """Vervang variabelen met bindings-waarden."""
    output = []
    for w in response:
        if w[0]=='?' and (w in bindings.keys()):
            output.append(bindings[w])
        else:
            output.append(w)
    return output

pattern = "i need a ?x".split()
input_  = "i need a vacation".split()
bindings = pat_match(pattern, input_)
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

response = sublis(bindings, 'what would it mean to you if you got ?x ?'.split())
print(' '.join(response)) # gebruik 'join' om van een lijst een string te maken

['i', 'need', 'a', '?x'] == ['i', 'need', 'a', 'vacation'] is True
Bindings = {'?x': 'vacation'}
what would it mean to you if you got vacation ?


Er is nog een laatste aanpassing aan de `pat_match` functie nodig om een bruikbare ELIZA te maken. Dit is een aanpassing om ELIZA geschikt te maken voor situaties waarin een variable gelijk is aan meerdere woorden. Dit wordt 'segment matching' genoemd. Om dit te realiseren wordt een segment variabele met het symbool `?*` geïntroduceerd. Hiermee wordt het volgende gedrag bereiken:

`pattern = '?*p need ?*x'
input_ = 'Mr Hulot and I need a vacation'
->
?p = 'Mr Hulot and I'
?x = 'a vacation'`

De uitdaging bij segment matching is om te bepalen hoeveel van de input moet worden toegekend aan de segment variabele. De oplossing hiervoor is om 'vooruit te kijken'. In de code hieronder wordt het resultaat van het vooruit kijken opgeslagen in `b2`. 

In [395]:
def pat_match(pattern, input_, bindings={}):
    """Match pattern against input in the context of the bindings"""
    if bindings==None:
        return None
    elif variable_p(pattern):
        return match_variable(pattern[0], input_[0], bindings)
    elif pattern == input_:
        return bindings
    elif segment_pattern_p(pattern):
        return segment_match(pattern, input_, bindings)
    elif atom(pattern) and atom(input_):
        if pattern == input_:
            return bindings
        else:
            return None
    elif consp(pattern) and consp(input_):
        return pat_match(rest(pattern), rest(input_), 
                         pat_match(first(pattern), first(input_), bindings))
    else:
        return None

def variable_p(x):
    """Is x a variable (a symbol beginning with '?')?"""
    return atom(x) and x[0][0] == '?' and x[0][1] != '*'

def match_variable(var, input_, bindings):
    """Does var match input_? Uses (or updates) and returns bindings."""
    if var[1] == '*':
        var = var[0] + var[2:]
        
    if var not in bindings.keys():
        bindings[var] = input_
        return bindings
    elif input_ == bindings[var]:
        return bindings
    return None


def segment_pattern_p(pattern):
    """Is this a segment matching pattern: (a symbol beginning with '?*')."""
    return consp(pattern) and first(pattern)[0].startswith("?*")


def segment_match(pattern, input_, bindings, start=0):
    """Match the segment pattern (?*var pat) against input."""

    var, pat = first(pattern), rest(pattern)
    if not pat:
        return match_variable(var[0], ' '.join(input_), bindings)
    
    pos = position(first(pat)[0], input_, start)
    if pos == None:
        return None

    b2 = pat_match(pat, input_[pos:], match_variable(var[0], ' '.join(input_[:pos]), dict(bindings)))

    if b2 == None:
        return segment_match(pattern, input_, bindings, pos+1)
    else:
        return b2

    
def position(item, input_, start):   
    """Zoek positie van een item in een lijst."""
    for i, w in enumerate(input_[start:]):
        if w == item:
            return start + i
    return None

In [396]:
pattern = '?*p need ?*x'.split()
input_  = 'Mr Hulot and I need a vacation'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

['?*p', 'need', '?*x'] == ['Mr', 'Hulot', 'and', 'I', 'need', 'a', 'vacation'] is True
Bindings = {'?p': 'Mr Hulot and I', '?x': 'a vacation'}


In [397]:
pattern = '?*x is a ?*y'.split()
input_  = 'what he is is a fool'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

['?*x', 'is', 'a', '?*y'] == ['what', 'he', 'is', 'is', 'a', 'fool'] is True
Bindings = {'?x': 'what he is', '?y': 'fool'}


In [398]:
pattern = '?*x a b ?*x'.split()
input_  = '1 2 a b a b 1 2 a b'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

['?*x', 'a', 'b', '?*x'] == ['1', '2', 'a', 'b', 'a', 'b', '1', '2', 'a', 'b'] is True
Bindings = {'?x': '1 2 a b'}


In [399]:
pattern = '?*x I need something'.split()
input_  = 'I need something'.split()
bindings = pat_match(pattern, input_, bindings={})
print("{} == {} is {}".format(pattern, input_, bindings!=None))
print("Bindings = {}".format(bindings))

['?*x', 'I', 'need', 'something'] == ['I', 'need', 'something'] is True
Bindings = {'?x': ''}


# ELIZA

Met de patroonherkenningsfunctie `pat_match` zoals die hierboven staat kan ELIZA gemaakt worden. Eerst wordt een database met regels gemaakt die ELIZA gebruikt om te reageren op de input van een gebruiker. Daarna wordt een lus geprogrameerd waarin de volgende stappen worden doorlopen:

1. vraag input van de gebruiker
2. zoek een regel waarvan het patroon past bij de input
3. als een regel gevonden is, reageer met een willekeurige reactie van die regel
4. als geen regel gevonden is, reageer met een willekeurige standaard reactie
5. ga naar 1

Hier onder staat een implementatie van Weizenbaum's Eliza (zie ook <a href="https://www.cse.buffalo.edu/~rapaport/572/S02/weizenbaum.eliza.1966.pdf">Weizenbaum's artikel uit 1966</a>).

In [ ]:
import random

# gebruik alleen kleine letters in het patroon gedeelte
rules = {
    "?*x hello ?*y" : [
        "How do you do. Please state your problem.",
    ],
    "?*x computer ?*y" : [
        "Do computers worry you?",
        "What do you think about machines?",
        "Why do you mention computers?",
        "What do you think machines have to do with your problem?",
    ],
    "?*x name ?*y" : [
        "I am not interested in names",
    ],
    "?*x sorry ?*y" : [
        "Please don't apologize",
        "Apologies are not necessary",
        "What feelings do you have when you apologize?",
    ],
    "?*x I remember ?*y" : [
        "Do you often think of ?y ?",
        "Does thinking of ?y bring you anything else to mind?",
        "What else do you remember?",
        "Why do you recall ?y right now?",
        "What in the present situation reminds you of ?y ?",
        "What is the connection between me and ?y",
    ],
    "?*x do you remember ?*y" : [
        "Did you think I would forget ?y ?",
        "Why do you think I should recall ?y now?",
        "What about ?y",
        "You mentioned ?y",
    ],
    "?*x i want ?*y" : [
        "What would it mean if you got ?y",
        "Why do you want ?y",
        "Suppose you got ?y soon",
    ],
    "?*x if ?*y" : [
        "Do you really think its likely that ?y",
        "Do you wish that ?y",
        "What do you think about ?y",
        "Really-- if ?y",
    ],
    "?*x I dreamt ?*y" : [
        "Really-- ?y ?",
        "Have you ever fantasied ?y while you were awake?",
        "Have you dreamt ?y before",
    ],
    "?*x dream about ?*y" : [
        "How do you feel about ?y in reality?",
    ],
    "?*x dream ?*y" : [
        "What does this dream suggest to you?",
        "Do you dream often?",
        "What persons appear in your dreams?",
        "Don't you believe that dream has to do with your problem?",
    ],
    "?*x my mother ?*y" : [
        "Who else in your family ?y",
        "Tell me more about your family",
    ],
    "?*x my father ?*y" : [
        "Your father",
        "Does he influence you strongly?",
        "What else comes t mind when you think of your father?",
    ],
    "?*x I am glad ?*y" : [
        "How have I helped you to be ?y",
        "What makes you happy just now",
        "Can you explain why you are suddenly ?y",
    ],
    "?*x I am sad ?*y" : [
        "I am sorry to hear you are depressed",
        "I'm sure it's not pleasant to be sad",
    ],
    "?*x are like ?*y" : [
        "What resemblance do you see?",
        "Could there really be some connection?",
        "How?",
    ],
    "?*x alike ?*y" : [
        "In what way?",
        "What similarities are there?",
    ],
    "?*x same ?*y" : [
        "What other connections do you see?",
    ],
    "?*x no ?*y" : [
        "Why not?",
        "You are a bit negative",
        "Are you just saying NO to be negative?",
    ],
    "?*x i was ?*y" : [
        "Were you really?",
        "Perhaps I already knew you were ?y",
        "Why do you tell me you were ?y now?",
    ],
    "?*x was I ?*y" : [
        "What if you were ?y ?", 
        "Do you think you were ?y",
        "What ould it mean if you were ?y",
    ],
    "?*x I am ?*y" : [
        "Do you believe you are ?y",
        "Would you want to be ?y",
        "You wish I would tell you you are ?y",
        "What would it mean if you were ?y",            
    ],
    "?*x am ?*y" : [
        "Why do you say 'AM'?",
        "I don't understand that",
    ],
    "?*x are you ?*y" : [
        "Why are you interested in whether I am ?y or not?",
        "Would you prefer if I weren't ?y ?",
        "Perhaps I am ?y in your fantasies"
    ],
    "?*x you are ?*y" : [
        "What makes you think I am ?y ?",
    ],
    "?*x because ?*y" : [
        "Is that the real reason?",
        "What other reasons might there be?",
        "Does that reason seem to explain anything else?",
    ],
    "?*x were you ?*y" : [
        "perhaps I was ?y",
        "What do you think?",
        "What if I had been ?y",
    ],
    "?*x I can't ?*y" : [
        "Maybe you could ?y now",
        "What if you could ?y ?",
    ],
    "?*x i feel ?*y" : [
        "Do you often feel ?y",
    ],
    "?*x i felt ?*y" : [
        "What other feelings do you have?",
    ],
    "?*x I ?*y you ?*z" : [
        "Perhaps in my fantasy we ?y each other",
    ],
    "?*x why don't you ?*y" : [
        "Should you ?y yourself?",
        "Do you believe I don't ?y",
        "Perhaps I will ?y in good time"
    ],
    "?*x yes ?*y" : [
        "You seem quite positive",
        "You are sure",
        "I understand",
    ],
    "?*x no ?*y" : [
        "Why not?",
        "You are being a bit negative",
        "Are you saying 'NO' just to be negative?",
    ],
    "?*x someone ?*y" : [
        "Can you be more specific?"
    ],
    "?*x everyone ?*y" : [
        "Surely not everyone",
        "Can you think of anyone in particular?",
        "Who for example?",
        "You are thinking of a special person?",
    ],
    "?*x always ?*y" : [
        "Can you think of a specific examples?",
        "When?",
        "What incident are you thinking of?",
        "Really-- always",
    ],
    "?*x what ?*y" : [
        "Why do you ask?",
        "Does that question interest you?",
        "What is it you really want to know?",
        "What do you think?",
        "What comes to your mind when you ask that?",
    ],
    "?*x perhaps ?*y" : [
        "You do not seem quite certain",
    ],
    "?*x are ?*y" : [
        "Did you think they might not be ?y",
        "Possibly they are ?y",
    ],
    "?*x ?y" : [
        "Very interesting",
        "I am not sure I understand you fully",
        "what does that suggest to you?",
        "Please continue",
        "Go on",
        "Do you feel strongly about discussing such things?"
    ],
    "[exit]" : [
        "bye bye",
    ],
}

default = [
    "Tell me more",
    "I see",
    "Can you tell me more",
]

def eliza(rules, default):
    input_ = ''
    while input_ != '[exit]':
        input_ = input('you> ')
        input_ = preprocess(input_, [',', '.', ';', ':', '!', '?'])
        response = use_eliza_rules(rules, default, input_)
        print('eliza> {}'.format(response))

        
def use_eliza_rules(rules, default, input_):
    for pattern, responses in rules.items():
        bindings = pat_match(pattern.split(), input_.split(), bindings={})
        if bindings != None:
            # switch viewpoint
            for k in bindings:
                bindings[k] = switch_viewpoint(bindings[k])
            response = sublis(bindings, responses[random.randint(0, len(responses)-1)].split())
            return ' '.join(response)
    return default[random.randint(0, len(default)-1)]


def preprocess(input_, special_chars):
    # verwijder hoofdletters
    input_ = input_.lower()
    
    # verwijder speciale karakters
    for c in special_chars:
        input_ = input_.replace(c, '')
        
    return input_

def switch_viewpoint(input_):
    viewpoints = {'i'  : 'you',
                  'you': 'i',
                  'my' : 'your',
                  'me' : 'you',
                  'am' : 'are'}
    
    return ' '.join([viewpoints[w] if w in viewpoints.keys() else w for w in input_.split()])


eliza(rules, default)

you> hello
eliza> How do you do. Please state your problem.
you> I want to start my own AI company
eliza> Why do you want to start your own ai company
you> Because I really love Artificial Intelligence
eliza> Is that the real reason?
you> I think I can help many people
eliza> Can you tell me more
you> With AI you can built new applications
eliza> Tell me more
you> Do you know what a chatbot is?
eliza> Why do you ask?
you> Chatbots are built using AI
eliza> Possibly they are built using ai
you> Do you know a chatbot?
eliza> Tell me more
you> You are a chatbot
eliza> What makes you think I am a chatbot ?
you> I programmed you
eliza> I see


# Hoe verder

Deze versie van ELIZA is nog steeds erg beperkt. Zo houdt ELIZA de context van een dialoog niet bij en heeft ze beperkte kennis. Dit kan aangepast worden door bijvoorbeeld ELIZA informatie van het Internet te laten gebruiken. 

ELIZA vormt ook een basis voor zogenaamde Expert Systemen.
